In [1]:
import torch
import random
from transformers import AutoTokenizer, AutoModelForSequenceClassification
path = "../model/Reranker"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForSequenceClassification.from_pretrained(path)

c:\ProgramData\anaconda3\envs\IR\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from utils import Data
data_processor = Data(root_path="../")
data_processor.read_in_memory()
data_processor.dataset.keys()

正在处理文件collection.sampled.tsv 读取文件的格式为('pid', 'passage')
正在处理文件train_sample_queries.tsv 读取文件的格式为('qid', 'query')
正在处理文件train_sample_passv2_qrels.tsv 读取文件的格式为('qid', 'mark', 'pid', 'rating')
正在处理文件val_2021_53_queries.tsv 读取文件的格式为('qid', 'query')
正在处理文件val_2021_passage_top100.txt 读取文件的格式为('qid', 'mark', 'pid', 'rank', 'score', 'sys_id')
正在处理文件val_2021.qrels.pass.final.txt 读取文件的格式为('qid', 'mark', 'pid', 'rating')
正在处理文件test_2022_76_queries.tsv 读取文件的格式为('qid', 'query')
正在处理文件test_2022_passage_top100.txt 读取文件的格式为('qid', 'mark', 'pid', 'rank', 'score', 'sys_id')
正在处理文件test_2022.qrels.pass.withDupes.txt 读取文件的格式为('qid', 'mark', 'pid', 'rating')


dict_keys(['collection.sampled', 'train_sample_queries', 'train_sample_passv2_qrels', 'val_2021_53_queries', 'val_2021_passage_top100', 'val_2021.qrels.pass.final', 'test_2022_76_queries', 'test_2022_passage_top100', 'test_2022.qrels.pass.withDupes'])

In [3]:
random_query_id = random.choice(tuple(data_processor.dataset["train_sample_queries"].keys()))
random_query = data_processor.dataset["train_sample_queries"][random_query_id]["query"]
random_query

'what states have integrated math in ccss'

In [4]:
answers = {
    v['rating']: data_processor.dataset["collection.sampled"][pid]['passage']
    for pid, v in data_processor.dataset["train_sample_passv2_qrels"].items()
    if v["qid"] == random_query_id
}
len(answers)

1

In [5]:
print("Query:", random_query)
print("Answers:", answers['1'])

Query: what states have integrated math in ccss
Answers: 'Jump to navigation Jump to search. Integrated mathematics is the term used in the United States to describe the style of mathematics education which integrates many topics or strands of mathematics throughout each year of secondary school.'


In [6]:
inputs = tokenizer(
    random_query,
    answers['1'],
    return_tensors='pt',
    padding=True,
    truncation=True,
    max_length=512
)
inputs

{'input_ids': tensor([[  101,  2054,  2163,  2031,  6377,  8785,  1999, 10507,  4757,   102,
          1005,  5376,  2000,  9163,  5376,  2000,  3945,  1012,  6377,  5597,
          2003,  1996,  2744,  2109,  1999,  1996,  2142,  2163,  2000,  6235,
          1996,  2806,  1997,  5597,  2495,  2029, 17409,  2015,  2116,  7832,
          2030, 14119,  1997,  5597,  2802,  2169,  2095,  1997,  3905,  2082,
          1012,  1005,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]])}

In [7]:
with torch.no_grad():
    outputs = model(**inputs)
outputs.logits

tensor([[-2.4394]])